# Phi4 Tool: Advanced Query and Data Processing Utility

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>

## Description

The Phi4 Tool is a powerful application designed for advanced query processing, designed to handle complex data interactions with precision and efficiency. Using sophisticated validation and dynamic query handling methods, it simplifies the management of user requests and external system integrations. Ideal for developers and businesses looking to optimize workflow automation and data processing, Phi4 Tool enhances accuracy and scalability while reducing the potential for errors in high-demand environments. This versatile tool integrates seamlessly into various systems to streamline operations and improve overall performance.

## Step 1: Boilerplate Setup

This step sets up the boilerplate code for the project. It includes:

- ### Import Statements:

    - `os`: Used for running shell commands and accessing environment variables.

    - `load_dotenv`: Loads environment variables from a `.env` file into the system for easy configuration.

    - `clear_output`: Clears the notebook's output to keep it clean after successful setup.



- ### Global Variables:

    - `requirements_installed`: Tracks whether dependencies are already installed.
    
    - `max_retries`: Limits how many times the code will retry installing dependencies in case of failure.
    
    - `REQUIRED_ENV_VARS`: Specifies the environment variables that must be present.



- ### `install_requirements` Function:

    - Uses the `os.system` command to run `pip install -r requirements.txt`.

    - If the installation fails, it retries up to `max_retries`.

    - If retries are exhausted, it exits the program with a failure code.



- ### `setup_env` Function:

    - Loads environment variables from `.env`.

    - Verifies the presence of each required variable using the `check_env` function.

    - Exits the program if any required environment variable is missing.



- ### Execution:

    - Calls `install_requirements` to install dependencies.

    - Calls `setup_env` to validate the environment.

    - Clears the output and confirms the setup is complete.



In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

## Step 2: Imports and Default Constants  

### **Imports**  

- **`from typing import Union`**  
  - Used to define type hints for variables or return values that can take on multiple types (e.g., `Union[BaseModel, None]`).  

- **`from pydantic import BaseModel`**  
  - Imports `BaseModel` from Pydantic, which is used to define and validate data models.  

- **`import json`**  
  - Provides utilities for working with JSON data, such as parsing and formatting.  

- **`import traceback`**  
  - Enables the capturing and printing of detailed stack traces in case of exceptions.  

- **`import ollama`**  
  - Presumably the library for interacting with the Ollama API, which facilitates communication with a language model.  

---

### **Default Configuration Variables**  

- **`DEFAULT_SYSTEM_PROMPT`**  
  - The system's default instruction for the assistant:  
    `"You are an intelligent assistant. You are helping the user with their query."`  

- **`DEFAULT_TEMPERATURE`**  
  - Sets the randomness level of the language model’s output. A value of `0.5` balances creativity and determinism.  

- **`DEFAULT_MAX_TOKENS`**  
  - Restricts the maximum number of tokens (words/pieces) in the model's response.  

- **`DEFAULT_OLLAMA_MODEL`**  
  - Specifies the model to use, here set to `"phi4"`.  

- **`DEFAULT_VERBOSE`** and **`DEFAULT_DEBUG`**  
  - Flags to control the verbosity (`True`) and debugging behavior (`True`) of the function.  

---

### **Function: build_dummy_pydantic_object**  

#### **Function Definition**  
- **Purpose**: Creates a "dummy" instance of the given Pydantic schema to understand its structure.  

- **`return schema()`**  
  - Initializes and returns an instance of the provided Pydantic schema. Useful for extracting the schema structure without requiring input data.  

---

### **Function: generate_object**  

#### **Function Definition**  
- **Purpose**: Accepts parameters like `prompt`, `response_model`, and configurations for system behavior, such as temperature and verbosity.  

- **`if verbose or debug:`**  
  - Logs the prompt for transparency if verbosity or debugging is enabled.  

- **`prompt_with_structured_output`**  
  - A formatted string containing the:  
    1. User prompt.  
    2. JSON representation of the expected response schema (from `build_dummy_pydantic_object`).  
    3. Instructions for the assistant to strictly adhere to the schema and JSON format.  

#### **Debug Block**  
- Logs detailed parameter configurations (`params`) when debugging is enabled.  

#### **`response = ollama.chat(...)`**  
- Sends the request to the Ollama API with:  
  1. The model to use.  
  2. The messages to structure the conversation (e.g., system and user prompts).  
  3. The format (`json`) for the expected response.  

#### **`response.message.content`**  
- Extracts the JSON content from the response.  

#### **Logging**  
- Logs the response received from the API for debugging or verbose output.  

#### **`json.loads(response_json)`**  
- Parses the JSON string (`response_json`) into a Python dictionary.  

#### **`response_model.model_validate(response_obj)`**  
- Validates the parsed JSON dictionary (`response_obj`) against the provided Pydantic response model.  

#### **Final Logs**  
- Logs the successful creation of the structured object, or detailed debugging information if enabled.  

---

### **Exception Handling**  

- **`except Exception as e:`**  
  - Captures any errors that occur during object generation.  

- **`traceback.print_exc()`**  
  - Prints the full stack trace for debugging purposes if an exception occurs.  

- **`return None`**  
  - Ensures the function returns `None` in case of failure, providing a predictable return type.  


In [19]:
## Ollama: Generate Object

from typing import Union
from pydantic import BaseModel
import json
import traceback
import ollama

DEFAULT_SYSTEM_PROMPT = (
    "You are an intelligent assistant. You are helping the user with their query."
)
DEFAULT_TEMPERATURE = 0.5
DEFAULT_MAX_TOKENS = 100
DEFAULT_OLLAMA_MODEL = "phi4"
DEFAULT_VERBOSE = True
DEFAULT_DEBUG = True


def build_dummy_pydantic_object(schema: BaseModel) -> BaseModel:
    """
    Build a dummy Pydantic object using the given schema.

    Args:
      schema: The Pydantic schema to build the object from

    Returns:
      BaseModel: The dummy Pydantic object
    """
    return schema()


def generate_object(
    prompt: str,
    response_model: BaseModel,
    system=DEFAULT_SYSTEM_PROMPT,
    model=DEFAULT_OLLAMA_MODEL,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    debug=DEFAULT_DEBUG,
    verbose=DEFAULT_VERBOSE,
) -> Union[BaseModel, None]:
    """Generates an object using the OpenAI API and given response model."""
    try:
        if verbose or debug:
            print(f"Generating object for prompt: {prompt}")

        prompt_with_structured_output = f"""
            Prompt: {prompt} 
            SCHEMA: {build_dummy_pydantic_object(response_model).model_dump_json()}

            INSTRUCTIONS: 
            - RESPOND IN JSON FORMAT. 
            - STRICTLY FOLLOW THE SCHEMA.
            - MAKE SURE TO INCLUDE ALL THE REQUIRED FIELDS.
            - DON'T INCLUDE ANY ADDITIONAL FIELDS.
            - INCASE YOU DON'T HAVE AN ANSWER FOR A FIELD, LEAVE IT EMPTY.
        """

        if debug:
            params = {
                "prompt": prompt_with_structured_output,
                "system": system,
                "temperature": temperature,
                "max_tokens": max_tokens,
                "model": model,
            }
            params = json.dumps(params, indent=2)
            print(f"Params: {params}")

        response = ollama.chat(
            model=model,
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": prompt_with_structured_output},
            ],
            format="json",
        )

        response_json = response.message.content  # Get the response content

        if verbose or debug:
            print(f"{model}: Response received successfully. 🎉")
            print(f"Response: {response_json}")

        response_obj = json.loads(response_json)
        response_structured = response_model.model_validate(response_obj)

        if verbose or debug:
            print("Object generated successfully. 🎉")

        if debug:
            print(f"EasyLLM Response: {response_json}")
        return response_structured
    except Exception as e:
        print(f"Failed to generate object. Error: {str(e)}")
        if debug:
            traceback.print_exc()
        return None

## Step 3: Generate and Validate Random Movie Data

### Import the Required Module  

#### **`from pydantic import BaseModel`**  

- Imports the `BaseModel` class from the **Pydantic** library.  

- Used to define and validate structured data models in Python.  


---

### Define the Movie Model  

#### **Class Definition**  

**`class Movie(BaseModel):`**  

- Creates a new Pydantic model named **`Movie`**.  

- Represents the structure of a movie object with specific attributes and default values.  


---

### **Model Fields**  

1. **`title: str = "Movie name"`**  
   - Field named **`title`** of type `str`.  
   - **Default Value**: `"Movie name"`.  
   - Represents the **movie's title**.  

2. **`year: int = 0`**  
   - Field named **`year`** of type `int`.  
   - **Default Value**: `0`.  
   - Represents the **release year** of the movie.  

3. **`genre: str = "movie genre"`**  
   - Field named **`genre`** of type `str`.  
   - **Default Value**: `"movie genre"`.  
   - Represents the **genre** of the movie.  

4. **`director: str = "movie director"`**  
   - Field named **`director`** of type `str`.  
   - **Default Value**: `"movie director"`.  
   - Represents the **director** of the movie.  

5. **`rating: float = 0.0`**  
   - Field named **`rating`** of type `float`.  
   - **Default Value**: `0.0`.  
   - Represents the **rating** of the movie.  

---

### Generate a Movie Object  

#### **Code Snippet**  
**`response = generate_object(...)`**  

- Calls the **`generate_object`** function to create a random movie object based on the **`Movie`** model schema.  

---

#### **Arguments Passed**  

1. **`prompt="Generate a random movie"`**  
   - Instructs the AI to generate a **random movie**.  

2. **`response_model=Movie`**  
   - Specifies the **`Movie`** schema to validate and structure the output.  

3. **`verbose=False`**  
   - Disables **verbose logging** to reduce output noise.  

4. **`debug=False`**  
   - Disables **debug information** for a cleaner console output.  

---

#### **Function Details**  
- Interacts with an AI model via the **`ollama`** library.  

- Uses the **prompt** and schema to generate a structured movie object.  

- Ensures the result adheres to the **`Movie`** schema.  


---

### Print the Generated Movie Object  

#### **Code Snippet**  

**`print(response.model_dump_json())`**  

- Calls the **`model_dump_json()`** method on the **response** object.  

- Converts the Pydantic **`BaseModel`** instance into a **JSON-formatted string**.  

- Outputs the JSON string to the console for:  

  - **Readability**.  

  - **Usability** in other applications or systems.  

---

### Summary  

1. **Structured Schema**  
   - Defines a structured schema for a **movie object** using **Pydantic**.  

2. **AI-Generated Object**  
   - Generates a random movie object using an AI model while ensuring compliance with the schema.  

3. **Readable Output**  
   - Prints the generated movie object in **JSON format** for easy readability and usability.  


In [ ]:
from pydantic import BaseModel


class Movie(BaseModel):
    title: str = "Movie name"
    year: int = 0
    genre: str = "movie genre"
    director: str = "movie director"
    rating: float = 0.0


response = generate_object(
    prompt="Generate a random movie",
    response_model=Movie,
    verbose=False,
    debug=False,
)

print(response.model_dump_json())

## Step 4: Tool Execution and Response Generation

### Imports and Setup  

- **`random`**: A Python module used to generate random numbers.  

- **`BaseModel`**: A class from Pydantic, used to define schemas and validate data structures.  

---

### Stock Price Function  

- **`get_stock_price(symbol: str)`**: A function that simulates fetching a stock's price.  

- **`random.randint(1, 200)`**: Generates a random stock price between 1 and 200.  

- **`return`**: Returns a formatted string with the stock symbol and the random price.  

---

### Tools Setup  

- **`tools`**: A list of dictionaries defining available tools.  

    - Each tool contains:  

      - **`name`**: The name of the tool.  

      - **`description`**: A brief explanation of the tool's functionality.  

      - **`function`**: The function to execute the tool's logic (e.g., **`get_stock_price`**).  

      - **`input`**: A description of the required input parameters.  

      - **`output`**: A description of the function's output.  

---

### Query Setup  

- **`query`**: The user request, which asks for the stock price of "AAPL".  

---

### Pydantic Schema  

- **`class ToolUseRequest(BaseModel)`**: Defines a Pydantic model named **`ToolUseRequest`** that inherits from **`BaseModel`**.  

    - **`tool_name`**: **`str`** = "tool name": A string field representing the tool's name with a default value "tool name".  

    - **`tool_input`**: **`dict`** = `{...}`: A dictionary field representing the input arguments for the tool, with a default set of arguments (arg1, arg2, info).  

---

### Prompt Setup  

- **`prompt`**: A string containing the instruction for generating the tool request.  

    - The **`query`** variable is inserted into the prompt to tell the assistant what the user asked.  

    - It instructs the assistant to check whether any tool is needed and to provide the input in the predefined schema format.  

    - It also tells the assistant to respond strictly using the defined schema, and not to invent any additional fields or types.  

    - This setup ensures that the input for tools and the output will strictly follow the format required by the **`ToolUseRequest`** schema.  

---

### Tool Execution Function  

- **`call_tool(tool_name, tool_input)`**: A function that takes the tool name and input parameters and executes the corresponding function.  

    - It iterates through the **`tools`** list and checks for a match on the **`tool_name`**.  

    - When it finds a match, it calls the associated tool's function (**`tool["function"]`**) and passes the required parameters (**`**tool_input`**).  

    - If no match is found, it returns **`"Tool not found."`**.  

---

### Generating Object  

- **`generate_object(...)`**: This function generates the structured request based on the provided prompt.  

    - **`prompt`**: The input query and instructions for the assistant (including the available tools).  

    - **`response_model`**: The model (**`ToolUseRequest`**) that the assistant's response must match.  

    - **`verbose` and `debug`**: Flags to enable detailed logging of the process.  

    - The function will output an object based on the assistant's response that contains the name of the tool to be used and the input parameters.  

---

### Handling the Tool Output  

- **`tool_output`**: A variable initialized to **`None`**, which will later store the result of the tool.  

    - The **`if`** condition checks whether a valid tool is specified.  

    - It ensures that **`response.tool_name`** is not null, empty, or the string "None".  

    - If a valid tool is specified, the function **`call_tool(response.tool_name, response.tool_input)`** is called to execute the tool and obtain the result.  

    - The result is printed and assigned to the **`tool_output`**.  

---

### Final Prompt with Tool Output  

- **`prompt`**: A final formatted string that includes the original query, the tool used, the input for that tool, and the resulting output.  

    - This will display the entire flow from the user's query to the tool's execution and the final result.  

---

### Summary of Code's Flow  

- **Setup**: Defines a list of available tools and a query asking for the stock price of "AAPL".  

- **Schema**: Defines a Pydantic schema (**`ToolUseRequest`**) for structuring the tool request.  

- **Generate Object**: Uses the **`generate_object`** function to generate the structured tool request based on the user's query.  

- **Tool Execution**: If the generated response contains a valid tool name, it executes the tool (**`call_tool`**).  

- **Final Output**: Outputs the tool's result, including the tool name, input, and output.  


In [ ]:
import random
from pydantic import BaseModel


def get_stock_price(symbol: str) -> str:
    price = random.randint(1, 200)
    return f"The current price of {symbol} is {str(price)}."


tools = [
    {
        "name": "Get Stock Price",
        "description": "Get the current price of a stock.",
        "function": get_stock_price,
        "input": [
            {
                "name": "symbol",
                "type": "str",
                "description": "The stock symbol.",
            }
        ],
        "output": {
            "return_type": "str",
            "description": "The current price of the stock.",
        },
    }
]

query = "Get the current price of AAPL."


class ToolUseRequest(BaseModel):
    tool_name: str = "tool name"
    tool_input: dict = {
        "arg1": "val1",
        "arg2": "val2",
        "info": "This object should contain the args for the tool and the arg names should correspond to the provided tool args.",
    }


prompt = f"""For the given query: {query}. 
    Tell me if you need any tools to be used and provide the input for the tool."
    These are the available tools: {tools}".
    If no tool needs to be used respond with null values.
    Respond strictly in the given schema format.
    Don't make up field names or types.
    Use the exact field names and types as given in the schema.
"""


def call_tool(tool_name: str, tool_input: dict):
    for tool in tools:
        if tool["name"] == tool_name:
            return tool["function"](**tool_input)
    return "Tool not found."


response = generate_object(
    prompt=prompt,
    response_model=ToolUseRequest,
    verbose=True,
    debug=True,
)

tool_output = None

if not (
    not response.tool_name
    or response.tool_name == "null"
    or response.tool_name == "None"
    or response.tool_name == ""
):
    result = call_tool(response.tool_name, response.tool_input)
    print("Tool Output:", result)
    tool_output = result

prompt = f"""Query: {query}
    TOOLS USED:
    Tool Name: {response.tool_name}
    Tool Input: {response.tool_input}
    Tool Output: {tool_output}
"""

## Conclusion

This code is designed to help manage user queries and connect with different external tools in a smooth and organized way. It uses a system called Pydantic to check that the information users provide is accurate and follows the required format. This helps avoid errors and ensures that the program works properly.

When a user asks a question, the code decides which external tool (like an API or service) is needed to answer the query and then runs that tool. The results are then returned in a clear, expected format.

The way the code is set up makes it easy to add new tools or services, which means it can be used for various tasks. For example, it could be applied in customer service systems, where the program automatically interacts with users and uses external services to help answer their questions or solve problems.

In short, this system allows the program to talk to different tools and services easily, making it efficient for automating tasks, answering queries, and processing information. It is designed to be flexible, reliable, and easy to expand for future needs.

---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>